# KNN (K-Nearest-Neighbors)

KNN is can be used for both classification and regression. In this method, a metric is defined as a scale for similarity based on some features or a combination of features. Then this can be used to find the class of a new item or a list of k-items which are close to it. 

### Aim 
The rating of the apps and its genre is derived. The method demostaretd in this example will help the app store for suggesting similar apps to the user when he is in a particular app's installation page.

### Data
The data consists of app store data from the google store, aim is to find similar apps given user has a app installed.

Data is Downloaded from Kaggle from the below link and author is Lavanya Gupta<br>
https://www.kaggle.com/lava18/google-play-store-apps/download

### Libraries used


In [1]:
import pylab as pl
import pandas as pd
import numpy as np
from scipy import spatial
import operator

### Data Loading


In [2]:
df = pd.read_csv("Data/googleplaystore.csv")
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


The genre column is a combined column which needs to be transformed to mulitple columns for processing
### Data Transformation

The generes are combined in the raw data which need to be transformed into a matrix format.

In [3]:
# Test code for tranformation
# string = 'Art & Design;Pretend Play'
# print (string.split(';'))
   
genreDict = set();

#df['GenresArray'] = df['Genres'].apply(lamda x: x.split(';'))
#k=0;
#df['Genres'].tolist
for index, row in df.iterrows():
    #k +=1
    #print(row['Genres'])
    row['GenresArray'] = row['Genres'].split(';');    
    df.at[index,'Installs'] =  int(''.join(filter(lambda x: x.isdigit(), row['Installs'])) )
    #df.at[index,'GenresArray']
    for genre in row['GenresArray']:
        if(genre not in genreDict):
            df[genre] = 0;            
            genreDict.add(genre)                  
        df.at[index,genre] = 1;         
        #print (genre)
        row[genre]=1
    #print(row['GenresArray'])
    #if (k>10):
    #   break;

### Adding an index for easy access
The ID column is added at the End, but it doesnt affect the functionality

In [4]:
df['appID'] = df.index
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Travel & Local,Tools,Personalization,Productivity,Parenting,Weather,News & Magazines,Maps & Navigation,Casino,appID
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0,Everyone,Art & Design;Pretend Play,...,0,0,0,0,0,0,0,0,0,1
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,5000000,Free,0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,2
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000,Free,0,Teen,Art & Design,...,0,0,0,0,0,0,0,0,0,3
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000,Free,0,Everyone,Art & Design;Creativity,...,0,0,0,0,0,0,0,0,0,4


### Normalization of app rating

The rating of the app is normalized, that is, the values are scaled between 0 and 1.<br>0 will be value for the app with lowest rating and 1 will be the value for the app with highest rating.

In [5]:
appRatings = pd.DataFrame(df['Rating'])
print(appRatings.dtypes)
#print(int(appRatings.at[0,'Reviews']))

normalizedAppRatings = appRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
normalizedAppRatings.head()

Rating    float64
dtype: object


,Rating
0,0.775
1,0.725
2,0.925
3,0.875
4,0.825


### Normalization of number of installs

The installs(number of downloads) of the app is normalized, that is, the values are scaled between 0 and 1.<br>0 will be value for the app with lowest number of users and 1 will be the value for the app with highest number of user.

In [6]:
appPopularity = pd.DataFrame(df['Installs'])
print(appPopularity.dtypes)
#print(int(appRatings.at[0,'Reviews']))

normalizedAppPopularity = appPopularity.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
normalizedAppPopularity.head()

Installs    object
dtype: object


,Installs
0,0.00001
1,0.00050
2,0.00500
3,0.05000
4,0.00010


### New Data Frame with normalized ratings and number of installs (popularity) of an app.

This data frame will have the normalized data which will be used for calculations.

In [7]:
appDict = {}

for index, row in df.iterrows():
    #print(row[0])
    appID = int(row[66])
    appName = row[0]
    genres = row[13:65]
    genres = map(int, genres)    
    genres=list(genres)
    #print(genres)
    #print(normalizedAppRatings.iloc[0]['Rating']);
    #print (row[2]);
    appDict[appID] = (appName, genres, normalizedAppRatings.iloc[appID]['Rating'], row[2],
                      normalizedAppPopularity.iloc[appID]['Installs'],row[5] )

#### Sample Data Structure

In [8]:
print(appDict[1])

('Coloring book moana', [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.72499999999999998, 3.9, 0.00050000000000000001, 500000)


### Compute Distance Function 
This function is designed to define a metric which essentially represent distance between two points based on value of features.
<br>
In this case, the distance is defined as sum of **3 values**, <br> 
one value is the cosine distance between **genres** two lists (**given more weigtage**), <br>
the second is the absolute difference between **ratings** of two apps and <br> 
the third is the **popularity** of the apps.
<br><br>
**Note 1:** Cosine function is used, because if values in the array are similar the value is low, and get higher as the differene increases. Value returned is always between 0 and 1 (normalized). <br>
**Note 2:** Weightage for the Genre is increased to have more similar apps



In [9]:
def ComputeDistance(a, b):      
    genresA = list(a[1])
    genresB = list(b[1])
    #print(genresA);
    #print(genresB);
    genreDistance = spatial.distance.cosine(genresA, genresB)
    ## Increase Weightage for the Genre to have more similar apps
    genreDistance = genreDistance * 2;
    ratingA=a[2]
    ratingB=b[2]
    ratingDistance = abs(ratingA - ratingB)
    popularityA = a[4]
    popularityB = b[4]
    #print("Popuralirty")
    #print(popularityA);
    #print(popularityB);
    popularityDistance = abs(popularityA - popularityB)
    #print(genreDistance);
    #print(popularityDistance);
    return genreDistance + ratingDistance + popularityDistance ;  

### Unit Test of the function
The distance between the apps at index 100 and 200 are calculated below using **ComputeDistance** Fn

In [10]:
distance = ComputeDistance(appDict[100], appDict[200])
print(appDict[100]) 
print('\n')
print(appDict[200])
print('\n')
print("The distance between two apps is : " + str(distance))

('Natural recipes for your beauty', [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.92500000000000004, 4.7, 0.0001, 100000)


('SuperLivePro', [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.82499999999999996, 4.3, 0.001, 1000000)


The distance between two apps is : 2.1009


### GetNeighbours Function

This function finds the app with the least distance from the app specified.<br>
it takes two inputs appID and number of neighbours and <br>
return a list of apps which have the least distance from the input app

In [11]:
def getNeighbors(appID, K):
    distances = []
    for app in appDict:
        if (app != appID):
            #print(appDict[appID])
            #print(appDict[app])
            dist = ComputeDistance(appDict[appID], appDict[app])
            distances.append((app, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

### List of K-Nearest Neighbours
The **Fn GetNeighbours** is going to be used for finding the similar apps of "Meme Generator"" and which are popular with the users who use the app.

In [12]:
appID=1001;
print ('K-nearest(k=5) neighbour of the app "' +  df['App'][appID] + '"');
print("with the rating of "+ str(df['Rating'][appID]) + " are :");
K = 5
avgRating = 0
neighbors = getNeighbors(appID, K)
for neighbor in neighbors:
    avgRating += appDict[neighbor][3]
    print ("Name of the app : " + appDict[neighbor][0] + " , Rating : " + str(appDict[neighbor][3]))


K-nearest(k=5) neighbour of the app "Meme Generator"
with the rating of 4.6 are :


C:\Users\lavis\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\scipy\spatial\distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Name of the app : Mandala Coloring Book , Rating : 4.6
Name of the app : ibis Paint X , Rating : 4.6
Name of the app : Anime Manga Coloring Book , Rating : 4.5
Name of the app : Sad Poetry Photo Frames 2018 , Rating : 4.5
Name of the app : HD Mickey Minnie Wallpapers , Rating : 4.7


### Comparison of the Average rating of the K-neighbours (5 neighbours) and the app used.


In [13]:
### Dividing by total number of neighbours    
avgRating /= K

print( "The average rating of the nearest neighbours is " + str(avgRating))
print( "The Rating of the app "+ str(df['Rating'][appID]))

The average rating of the nearest neighbours is 4.58
The Rating of the app 4.6


#### Observation
The ratings of the apps is also similar as the app "Meme Generator".
### Results

The list of the 5 apps which are similar to "Meme Generator"
with the rating of 4.6 are :

Name of the app : Mandala Coloring Book , Rating : 4.6 <br>
Name of the app : ibis Paint X , Rating : 4.6<br>
Name of the app : Anime Manga Coloring Book , Rating : 4.5<br>
Name of the app : Sad Poetry Photo Frames 2018 , Rating : 4.5<br>
Name of the app : HD Mickey Minnie Wallpapers , Rating : 4.7<br>

### Conclusion

The Current method demostarted can find the similar apps for the users who have installed any particular app using the K nearest neighbours method.